In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.5 MB/s eta 0:00:00


In [ ]:
 !pip install datasets sentencepiece optuna

In [ ]:
pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments,EarlyStoppingCallback
import torch
from transformers import Seq2SeqTrainer
from torch.utils.data import Dataset, DataLoader
from itertools import product
import optuna
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

nlp = spacy.load("en_core_web_sm")
# download necessary resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:

with open('/content/drive/MyDrive/MTP Project/t5_data/true_300_ent_attr.json') as jsonfile:
    train_data = json.load(jsonfile)
from sklearn.model_selection import train_test_split

# Assuming 'data' is your list of dictionaries
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42, stratify=None)

# train_data contains the training samples
# val_data contains the validation samples

print(f"Total Training dataset samples : {len(train_data)}")
print(f"Total validation set samples : {len(val_data)}")


df_train = pd.DataFrame(train_data)
df_val=pd.DataFrame(val_data)

Total Training dataset samples : 270
Total validation set samples : 30


In [ ]:
train_data[0]
#from annotated 300 samples, prepare the input and output dataset
#for input , append sentence +What are the entity and attribute related with quantity  {quantity value}'
#for output , take the {entity_value} <sep> { attribute value }

{'input': 'The road distance from Mörön to Ulaanbaatar is 690 km . What are the entity and attribute related with quantity 690 km?',
 'output': 'Mörön to Ulaanbaatar <sep> road distance'}

In [ ]:
def preprocess(row,max_input_len,max_target_len,tok):
    tok=tok
    input_str = row['input']
    target_str = row['output']
    tokenized_inputs = tok(input_str, max_length=max_input_len, truncation=True)
    tokenized_targets = tok(text_target=target_str, max_length=max_target_len, truncation=True)
    tokenized_inputs['labels'] = tokenized_targets['input_ids']
    return tokenized_inputs
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.data[idx]['input_ids'],
            'attention_mask': self.data[idx]['attention_mask'],
            'labels': self.data[idx]['labels'],

        }

In [ ]:
#load model parameters
with open("/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/final_params.json") as f:
  params=json.load(f)
print(params)

{'dropout_rate': 0.02154867810249722, 'input_seq_length': 256, 'learning_rate': 0.00022763326997820923, 'num_train_epochs': 24, 'output_seq_length': 14, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.0013437781293502878}


In [ ]:
model_name = "t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,dropout_rate=params['dropout_rate'])
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_tokens(["<sep>"])
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
train_dataset = df_train.apply(lambda row: preprocess(row,params['input_seq_length'],params['output_seq_length'],tokenizer), axis=1)
val_dataset = df_val.apply(lambda row: preprocess(row, params['input_seq_length'],params['output_seq_length'],tokenizer), axis=1)
train_dataset=train_dataset.to_list()
val_dataset=val_dataset.to_list()
train_dataset = MyDataset(train_dataset)
val_dataset = MyDataset(val_dataset)

In [ ]:
dir='/content/drive/MyDrive/MTP Project/out_attr/cv_300_ent_attr/checkpoints/'
training_args = Seq2SeqTrainingArguments(

  output_dir=dir,
  evaluation_strategy="epoch",
  save_strategy="epoch",
   logging_strategy='epoch',
  learning_rate=params['learning_rate'],
  per_device_train_batch_size=params['per_device_train_batch_size'],
  per_device_eval_batch_size=params['per_device_eval_batch_size'],
  weight_decay=params['weight_decay'],
  save_total_limit=1,
  num_train_epochs=params['num_train_epochs'],
  predict_with_generate=True,
  load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False




)

In [ ]:
def post_process(text):
  text = text.lower()
  tokens = nltk.word_tokenize(text)
  for i in range(len(tokens)):
     w=tokens[i]
     w=w.replace(" ","")
     tokens[i]=w
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [word for word in tokens if word not in stop_words]
  preprocessed_text = ' '.join(tokens)
  doc = nlp(preprocessed_text)
  lemmatized_text = " ".join([token.lemma_ for token in doc])
  text = re.sub('[^a-zA-Z0-9\s]', '',  lemmatized_text)
  tokens = nltk.word_tokenize(text)
  return tokens
#post processing for entity
def preprocess_ent(text):
  # lowercase the text
   text = text.lower()
   text=re.sub(r'\b(a|an|the)\b', '', text)
   text=text.replace("'s","")
   text=text.replace(" 's","")


# tokenize the text
   tokens = nltk.word_tokenize(text)
   for i in range(len(tokens)):
     w=tokens[i]

     w=w.replace(" ","")
     tokens[i]=w

   return tokens

In [ ]:
def compute_metrics(eval_pred):
  tokenizer = AutoTokenizer.from_pretrained("t5-base")
  tokenizer.add_tokens(["<sep>"])
  preds, labels = eval_pred
  prec,recall,ent_p,ent_r=[],[],[],[]
  if isinstance(preds, tuple):
       preds = preds[0]
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  for i in range(len(decoded_preds)):
        pred_text=decoded_preds[i]
        label_text=decoded_labels[i]
        #seperate entity and attribute for predicted text
        if('<sep>' in pred_text ):
          pred_ent=pred_text.split('<sep>')[0].strip()
          pred_attr=pred_text.split('<sep>')[1].strip()

        else:

          pred_ent= pred_text.strip()
          pred_attr=''
        #seperate entity and attribute for labelled text
        if('<sep>' in label_text ):
           label_ent=label_text.split('<sep>')[0].strip()
           label_attr=label_text.split('<sep>')[1].strip()
        else:
          label_ent= label_text.strip()
          label_attr=''
        #find the predicted and labelled entity
        pred_ent=preprocess_ent( pred_ent)
        label_ent=preprocess_ent( label_ent)
        #find the predicted and labelled attribute
        pred_tokens= post_process(pred_attr)
        label_tokens= post_process(label_attr)


   # for attribute

         # calculate overlap
        overlap = len(set(pred_tokens).intersection(set(label_tokens)))
        # calculate relaxed precision
        if(len(pred_tokens)!=0):
           rp = overlap / len(pred_tokens)
        else:
           rp=0
        prec.append(rp)
        if(len(label_tokens)!=0):
    # calculate relaxed recall
           rr = overlap / len(label_tokens)
        else:
             rr=0
        recall.append(rr)

    #for entity
        # calculate overlap
        overlap = len(set(pred_ent).intersection(set(label_ent)))
          # calculate relaxed precision
        if(len(pred_ent)!=0):
           rp = overlap / len(pred_ent)
        else:
           rp=0
        ent_p.append(rp)
        if(len(label_ent)!=0):
    # calculate relaxed recall
           rr = overlap / len(label_ent)
        else:
             rr=0
        ent_r.append(rr)


  precision,recall=np.mean(prec),np.mean(recall)
  f1=(2*precision*recall )/ (precision+recall)

  ent_precision,ent_recall=np.mean(ent_p),np.mean(ent_r)
  f1_ent= (2*ent_precision* ent_recall) / (ent_precision+ent_recall)




  res={'Attribute Relaxed Precision':precision,'Attribute Relaxed Recall':recall,'Attribute F1-score':f1 , 'Entity Relaxed Precision':ent_precision,'Entity Relaxed Recall':ent_recall,'Entity F1-score':f1_ent}

  return res




In [ ]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Attribute relaxed precision,Attribute relaxed recall,Attribute f1-score,Entity relaxed precision,Entity relaxed recall,Entity f1-score
1,5.007000,3.081709,0.000000,0.000000,nan,0.716667,0.722222,0.719434
2,2.000500,1.302673,0.033333,0.033333,0.033333,0.462143,0.850000,0.598748
3,0.868700,1.243939,0.558333,0.569444,0.563834,0.857143,0.866667,0.861878
4,0.586700,1.170450,0.594444,0.563889,0.578764,0.851587,0.861111,0.856323
5,0.362100,1.063886,0.511111,0.525000,0.517962,0.861111,0.866667,0.863880
6,0.256800,1.162171,0.558333,0.513889,0.535190,0.861111,0.833333,0.846995
7,0.197500,1.165894,0.563889,0.547222,0.555431,0.861111,0.850000,0.855519
8,0.145200,1.109897,0.600000,0.569444,0.584323,0.827778,0.816667,0.822185
9,0.126200,1.158764,0.577778,0.608333,0.592662,0.827778,0.816667,0.822185
10,0.094500,1.078162,0.611111,0.597222,0.604087,0.861111,0.850000,0.855519


TrainOutput(global_step=408, training_loss=0.43421060255929533, metrics={'train_runtime': 647.8133, 'train_samples_per_second': 10.003, 'train_steps_per_second': 0.63, 'total_flos': 552408062699520.0, 'train_loss': 0.43421060255929533, 'epoch': 24.0})

In [ ]:
logs=trainer.state.log_history

In [ ]:
import json
with open('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/logs_new.json','w') as f:
  json.dump(logs,f)


In [ ]:
#save the pretrained model
model.save_pretrained('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_model')

In [ ]:
#save tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer')

('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer/spiece.model',
 '/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/t5_attr_tokenizer/tokenizer.json')

In [ ]:
train_loss,val_loss,prec,recall,f1={},{},{},{},{}
for i in logs:

  if 'loss' in i:
    train_loss[i['epoch']]=i['loss']

  elif 'eval_loss' in i:
    val_loss[i['epoch']]=i['eval_loss']
    prec[i['epoch']]= i['eval_Relaxed Precision']
    recall[i['epoch']]=i['eval_Relaxed Recall']
    f1[i['epoch']]=i['eval_f1']


In [ ]:
#100 samples with 25 of each seq len
with open('/content/drive/MyDrive/MTP Project/t5_data/test_data_100_seperate.json' ) as f:
  rule_test=json.load(f)
len(rule_test)

100

In [ ]:
rule_test

In [ ]:
test_json=[]
for i in rule_test:
  d={}
  inp=i['sentence']+" "+ f"What are the entity and attribute related with quantity {i['quantity']}?"

  outt=i['entity']+' <sep> '+i['attribute']
  d['input']=inp
  d['output']=outt
  d['entity']=i['entity']
  d['quantity']=i['quantity']
  d['sen']=i['sentence']
  test_json.append(d)
len(test_json)

100

In [ ]:
test_json[0]

{'input': 'The total length of this LPG pipeline is 1415 km . What is the attribute related to entity this LPG pipeline with quantity 1415 km?',
 'output': 'total length',
 'entity': 'this LPG pipeline',
 'quantity': '1415 km',
 'sen': 'The total length of this LPG pipeline is 1415 km .'}

In [ ]:
#store result as list of dictionaries with keys as sentence,entity,attribute
res=[]
for i in range(len(test_json)):

  d=test_json[i]

  input=d['input']

  tokenized_inputs= tokenizer(input, max_length=256, truncation=True,return_tensors='pt')
  tokenized_inputs=tokenized_inputs.to('cuda')
  output_sequence = model.generate(**tokenized_inputs
                                 )

# decode the output sequence into text using the tokenizer
  #d=rule_test[i]
  u={}
  print(d['sen'])
  print("------------------------------Rule based result-------------------------------")
  print(f" Entity : {d['entity']}")
  print(f" Quantity : {d['quantity']} ")
  print(f" Attribute : {d['output']} ")
  print("-------------------------------- T5 Result------------------------------------")
  output_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
  print(f" Attribute : {output_text}")
  print('\n\n')
  u['sentence']=d['sen']
  #seperate entity and attribute in predicted text
  if('<sep>' in output_text):
   ent=output_text.split('<sep>')[0].strip()
   attr=output_text.split('<sep>')[1].strip()
  else:
    ent=output_text
    attr=''

  u['attribute']=attr
  u['entity']=ent
  res.append(u)

In [ ]:
import json
with open ('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/test_25samples_prediction.json','w') as f:
  json.dump(res,f)


In [ ]:
#test data according to dataset distribution
with open('/content/drive/MyDrive/MTP Project/t5_data/new_test_data_100.json' ) as f:
  rule_test=json.load(f)
len(rule_test)

100

In [ ]:
test_json=[]
for i in rule_test:
  d={}
  inp=i['sentence']+" "+ f"What are the entity and attribute related with  quantity {i['quantity']}?"

  outt=i['entity'] + ' <sep> '+i['measuring attribute']
  d['input']=inp
  d['output']=outt
  d['entity']=i['entity']
  d['quantity']=i['quantity']
  d['sen']=i
  test_json.append(d)
len(test_json)

100

In [ ]:
test_json[0]

{'input': 'According to the United States Census Bureau , the CDP covers an area of , 99.69 % of it land , and 0.31 % of it water . What are the entity and attribute related with  quantity 99.69 %?',
 'output': 'the CDP <sep> area',
 'entity': 'the CDP',
 'quantity': '99.69 %',
 'sen': 'According to the United States Census Bureau , the CDP covers an area of , 99.69 % of it land , and 0.31 % of it water .'}

In [ ]:
res=[]
for i in range(len(test_json)):

  d=test_json[i]

  input=d['input']

  tokenized_inputs= tokenizer(input, max_length=256, truncation=True,return_tensors='pt')
  tokenized_inputs=tokenized_inputs.to('cuda')
  output_sequence = model.generate(**tokenized_inputs
                                 )

# decode the output sequence into text using the tokenizer
  #d=rule_test[i]
  u={}
  print(d['sen'])
  print("------------------------------Rule based result-------------------------------")
  print(f" Entity : {d['entity']}")
  print(f" Quantity : {d['quantity']} ")
  print(f" Attribute : {d['output']} ")
  print("-------------------------------- T5 Result------------------------------------")
  output_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
  print(f" Attribute : {output_text}")
  print('\n\n')
  u['sentence']=d['sen']
  if('<sep>' in output_text):
   ent=output_text.split('<sep>')[0].strip()
   attr=output_text.split('<sep>')[1].strip()
  else:
    ent=output_text
    attr=''
  u['attribute']=attr
  u['entity']=ent
  res.append(u)

In [ ]:
import json
with open ('/content/drive/MyDrive/MTP Project/out_ent_attr/cv_300_ent_attr/test_data_distribution.json','w') as f:
  json.dump(res,f)
